In [ ]:
import pandas as pd
import numpy as np
import random

from itertools import cycle

Порой возникает необъодимость протестировать код. Для этого полезно уметь генерировать синтетические данные. В этом ноутбуке я привел ряд практик, которые позволят делать это быстро и легко

# Полезные ресурсы:
https://towardsdatascience.com/generating-fake-data-with-pandas-very-quickly-b99467d4c618

## Создание данных в ручную

In [ ]:
df = pd.DataFrame({
    'Name':['Tom', 'Brad', 'Kyle', 'Jerry'],
    'Age':[20, 21, 19, 18],
    'Height' : [6.1, 5.9, 6.0, 6.1]
})

df

Такой способ подойдет, если нужно быстро проверить функциональность, которая не требует большого разнообразия  и объема данных. И даже в этом случае 

# Готовые шаблоны
Но долго считается, надо бы оптимизировать

### Шаблон 1

In [ ]:
N = 500

df = pd.DataFrame({
    'name': [],
    'val1': [],
    'val2': [],
    'val3': []
})

### Шаблон 2

Этот вариант исполняется слишком долго

In [ ]:
N = 300

datetimes = pd.Series(pd.date_range(start='2021-07-01',end='2021-07-30', freq='S'))
dates = datetimes.dt.date.unique().tolist()
times = datetimes.dt.time.unique().tolist()

groups_list = ['A', 'B', 'C']
subgroup_list = ['a', 'b', 'c' , 'd']

value_min = 1
value_max = 100

df = pd.DataFrame({
    'date': [np.random.choice(dates, 1, replace=True)[0] for i in range(N)],
    'time': [np.random.choice(times, 1, replace=True)[0] for i in range(N)],
    'group': [np.random.choice(groups_list, 1, replace=True)[0] for i in range(N)],
    'sub_group': [np.random.choice(subgroup_list, 1, replace=True)[0] for i in range(N)],
    'value': np.random.sample(range(value_min, value_max), N)
}).sort_values(['date', 'time'])

# Мелкие генераторы

### Сгенерировать имена

In [ ]:
from faker.providers.person.en import Provider

def random_names(name_type, size):
    """
    Generate n-length ndarray of person names.
    name_type: a string, either first_names or last_names
    """
    names = getattr(Provider, name_type)
    return np.random.choice(names, size=size)

### Сгенерировать пол

In [ ]:
def random_genders(size, p=(0.49, 0.49, 0.01, 0.01)):
    """Generate n-length ndarray of genders."""
    gender = ("M", "F", "O", "")
    return np.random.choice(gender, size=size, p=p)

### Сгенерировать дату

In [ ]:
def random_dates(start, end, size):
    """
    Generate random dates within range between start and end.    
    Adapted from: https://stackoverflow.com/a/50668285
    """
    # Unix timestamp is in nanoseconds by default, so divide it by
    # 24*60*60*10**9 to convert to days.
    divide_by = 24 * 60 * 60 * 10**9
    start_u = start.value // divide_by
    end_u = end.value // divide_by
    return pd.to_datetime(np.random.randint(start_u, end_u, size), unit="D")

In [ ]:
size = 100  
df = pd.DataFrame(columns=['First', 'Last', 'Gender', 'Birthdate'])
df['First'] = random_names('first_names', size)
df['Last'] = random_names('last_names', size) 
df['Gender'] = random_genders(size)
df['Birthdate'] = random_dates(start=pd.to_datetime('1940-01-01'), end=pd.to_datetime('2008-01-01'), size=size)

df.head()

# Блошие генераторы

In [ ]:
def generate_fake_dataframe(size, cols, col_names = None, intervals = None, seed = None):
    
    categories_dict = {'animals': ['cow', 'rabbit', 'duck', 'shrimp', 'pig', 'goat', 'crab', 'deer', 'bee', 'sheep', 'fish', 'turkey', 'dove', 'chicken', 'horse'],
                       'names'  : ['James', 'Mary', 'Robert', 'Patricia', 'John', 'Jennifer', 'Michael', 'Linda', 'William', 'Elizabeth', 'Ahmed', 'Barbara', 'Richard', 'Susan', 'Salomon', 'Juan Luis'],
                       'cities' : ['Stockholm', 'Denver', 'Moscow', 'Marseille', 'Palermo', 'Tokyo', 'Lisbon', 'Oslo', 'Nairobi', 'Río de Janeiro', 'Berlin', 'Bogotá', 'Manila', 'Madrid', 'Milwaukee'],
                       'colors' : ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'purple', 'pink', 'silver', 'gold', 'beige', 'brown', 'grey', 'black', 'white']
                      }
    
    default_intervals = {"i" : (0,10), "f" : (0,100), "c" : ("names", 5), "d" : ("2020-01-01","2020-12-31")}
    rng = np.random.default_rng(seed)
    

    first_c = default_intervals["c"][0]
    categories_names = cycle([first_c] + [c for c in categories_dict.keys() if c != first_c])
    default_intervals["c"] = (categories_names, default_intervals["c"][1])
    
    if isinstance(col_names,list):
        assert len(col_names) == len(cols), f"The fake DataFrame should have {len(cols)} columns but col_names is a list with {len(col_names)} elements"
    elif col_names is None:
        suffix = {"c" : "cat", "i" : "int", "f" : "float", "d" : "date"}
        col_names = [f"column_{str(i)}_{suffix.get(col)}" for i, col in enumerate(cols)]

    if isinstance(intervals,list):
        assert len(intervals) == len(cols), f"The fake DataFrame should have {len(cols)} columns but intervals is a list with {len(intervals)} elements"
    else:
        if isinstance(intervals,dict):
            assert len(set(intervals.keys()) - set(default_intervals.keys())) == 0, f"The intervals parameter has invalid keys"
            default_intervals.update(intervals)
        intervals = [default_intervals[col] for col in cols]
        
        
    df = pd.DataFrame()
    for col, col_name, interval in zip(cols, col_names, intervals):
        
        if interval is None:
            interval = default_intervals[col]
        assert (len(interval) == 2 and isinstance(interval, tuple)) or isinstance(interval, list), f"This interval {interval} is neither a tuple of two elements nor a list of strings."
        
        if col in ("i","f","d"):
            start, end = interval
        if col == "i":
            df[col_name] = rng.integers(start, end, size)
        elif col == "f":
            df[col_name] = rng.uniform(start, end, size)
        
        elif col == "c":
            if isinstance(interval, list):
                categories = np.array(interval)
            else:
                cat_family, length = interval
                if isinstance(cat_family, cycle):
                    cat_family = next(cat_family)
                assert cat_family in categories_dict.keys(), f"There are no samples for category '{cat_family}'. Consider passing a list of samples or use one of the available categories: {categories_dict.keys()}"
                categories = rng.choice(categories_dict[cat_family], length, replace = False, shuffle = True)
            df[col_name] = rng.choice(categories, size, shuffle = True)
        
        elif col == "d":
            df[col_name] = rng.choice(pd.date_range(start, end), size)
            
    return df       

In [ ]:
generate_fake_dataframe(size = 1000, cols =  "cififficcd")

In [ ]:
df1 = generate_fake_dataframe(
    size = 10, 
    cols = "cccfd", 
    col_names=["name", "pet", "city","height", "birthday"],
    intervals = {"f" : (1.72,1.95), "d" : ("1996-01-01","1996-12-31")},
    seed=42
)

df1

In [ ]:
df2 = generate_fake_dataframe(
  size = 30, 
  cols = "cicffcd", 
  col_names = ["user", "age", "residence", "weight","height", "pet", "registered"],
  intervals = [("names",15), (18,25),("cities", 15), (73.2,95.0),
               (1.65,1.95), ("animals", 11), None],
  seed = None)

df2.head(10)

In [ ]:
df3 = generate_fake_dataframe(
    size = 20,
    cols = "cfcc",
    col_names = ["runner", "total_time", "skill", "adult"],
    intervals = [
        ("names", 15),
        (70,100),
        ["stamina", "endurance", "speed"],
        [True, False],
    ]
)

df3.head(10)

In [ ]:
df4 = generate_fake_dataframe(
    size = 20,
    cols = "ifccc",
    col_names = ["id", "target", "one_hot_1", "one_hot_2", "one_hot_3"],
    intervals = {"c" : [0,1]}
)

df4